In [1]:
import sys
sys.path.append('C:/ProgramData/Anaconda3/envs/tf-gpu/Lib/site-packages')
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, LeakyReLU, BatchNormalization
from keras import losses

from tensorflow.python.keras.callbacks import TensorBoard
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

QUANDL_API_KEY = 'wbLh97Ei-rW9ddbz7jHN'
MODEL_OUTPUT_SIGNAL_CUTOFF = 0.6

dnn_model = None
rf_model = None

#Get constants
%run ./CONSTANTS.ipynb
PERIODS = GET_PERIODS()

Using TensorFlow backend.


In [2]:
%load_ext zipline

ImportError: cannot import name 'RLock'

In [ ]:
%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.api import symbol, order, record

def initialize(context):
    context.i = 0
    context.asset = symbol('A')
    
    #Train and store our models
    global dnn_model
    global rf_model
    
    %run ./Train_Ensemble.ipynb
    (dnn_model, rf_model) = train_given_symbol(symb)
                                               
                                               
    
def handle_data(context, data):
    context.i += 1
    if context.i < 300:
        return
    
    #Get the data from our
    price_history = data.history(context.asset, fields=["open", "high", "low", "close", "volume"], bar_count=PERIODS, frequency="1d")
    
    #Calculate indicators for the rows
    %run ./Request.ipynb
    data_df_with_indicators = calculate_and_append_indicators(ohlc_df)
    
    #Construct one input row from the dataframe
    %run ./Train_Ensemble.ipynb
    input_np_arr = period_expand_np_arr(data_df_with_indicators.values)
    
    #Predict on the row
    rf_prediction = rf_model.predict(input_np_arr)
    dnn_prediction = dnn_model.predict_on_batch(input_np_arr)
    
    #Set our order according to the models' prediction
    if rf_prediction > MODEL_OUTPUT_SIGNAL_CUTOFF and dnn_prediction > MODEL_OUTPUT_SIGNAL_CUTOFF:
        order_target(context.asset, 100)
    else:
        order_target(context.asset, 0)
    
    #Save values for later inspection
    record(stk = data.current(context.asset, 'price'), rf = rf_prediction, dnn = dnn_prediction)

                                               
                                               
                                               
def analyze(context, perf):
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('portfolio value in $')

    ax2 = fig.add_subplot(212)
    perf['stk'].plot(ax=ax2)
    perf[['rf', 'dnn']].plot(ax=ax2)